# GJK

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K
import numpy as np

In [ ]:
from pkg.tf_transform import *
from pkg.tf_robot import *
from pkg.constraint import *
from pkg.info import *
from pkg.tf_utils import *
from pkg.rotation_utils import *
from pkg.utils import *

In [ ]:
import matplotlib.pyplot as plt
import time

## for test

In [ ]:
def get_facepoints(verts, faces, N_fcs=20): # (fc, vtx, ax)
    global_save['verts'] = verts
    global_save['faces'] = faces
    return np.concatenate([np.array([verts[s1f-1] for s1f in faces]), np.zeros((N_fcs-len(faces),3,3))],axis=0)

def pad_vertex(verts, N_vtx):
    l_v = len(verts)
    assert l_v<N_vtx, "vertex more than N_vtx"
    return np.concatenate([verts, np.zeros((N_vtx-l_v,3))],axis=0).astype('float32')

In [ ]:
gtimer = GlobalTimer()

In [ ]:
N_vtx = 20

In [ ]:
s1coords = np.loadtxt("s1.csv", delimiter=",")
s2coords = np.loadtxt("s2.csv", delimiter=",")

In [ ]:
s1faces = np.loadtxt("s1face.csv", delimiter=",",dtype='int')
s2faces = np.loadtxt("s2face.csv", delimiter=",",dtype='int')

In [ ]:
iterations = 6

In [ ]:
s1verts = pad_vertex(s1coords.astype('float32'), N_vtx) # get_facepoints(s1coords, s1faces,N_fcs).astype('float32')
s2verts = pad_vertex(s2coords.astype('float32'), N_vtx) # get_facepoints(s2coords, s2faces,N_fcs).astype('float32')

In [ ]:
S1Rot = np.array([
        [0.99847,0.016541,-0.052829,],
        [-0.014912,0.99941,0.031073,],
        [0.053311,-0.030238,0.99812,],
        ], dtype=np.float32)
S2Rot = np.array([
        [0.99752,0.065339,-0.026294,],
        [-0.063664,0.99616,0.060141,],
        [0.030123,-0.058318,0.99784,],
        ], dtype=np.float32)

In [ ]:
s1points = tf.matmul(s1verts, np.transpose(S1Rot))+3
s2points = tf.matmul(s2verts, np.transpose(S2Rot))-3

In [ ]:
FX1 = s1points
FX2 = s2points

## original_remove_face

In [ ]:
global_save = {}
@tf.function
def getFarthestInDir(FX, v):
    dotted = K.sum(tf.multiply(FX,v),axis=-1)
    IdxSet = tf.expand_dims(tf.argmax(dotted, axis=-1),axis=-1)
    point=tf.gather(params = FX, indices=IdxSet)
    return point

@tf.function
def support(FX1, FX2, v):
    point1 = getFarthestInDir(FX1, v)
    point2 = getFarthestInDir(FX2, -v)
    return point1 - point2

@tf.function
def pickLineTF(v, FX1, FX2):
    b= support(FX2, FX1, v)
    a= support(FX2, FX1, -v)
    return a, b

@tf.function
def _loop_PickTriangle(a, b, c, flag, FX1, FX2):
    ab = b-a;
    ao = -a;
    ac = c-a;
    abc = tf.linalg.cross(ab,ac)
    abp = tf.linalg.cross(ab,abc)
    acp = tf.linalg.cross(abc,ac)
    abpo = tf.greater(K.sum(abp*ao), 0)
    acpo = tf.greater(K.sum(acp*ao), 0)
    a,b,c,flag = tf.case(
        [
            (abpo, lambda: (support(FX2, FX1,abp),a,b,False)),
            (acpo, lambda: (support(FX2, FX1,acp),a,c,False))
        ], default=lambda: (a,b,c,True))
    return a,b,c,flag, FX1, FX2

@tf.function
def _cond_PickTriangle(a, b, c, flag, FX1, FX2):
    return tf.logical_not(flag)
    

@tf.function
def PickTriangleTF(a, b, FX1, FX2, IterationAllowed=6):
    flag = False

    # First try:
    ab = b-a
    ao = -a
    v = tf.linalg.cross(tf.linalg.cross(ab,ao),ab) # v is perpendicular to ab pointing in the general direction of the origin
    c = b
    b = a
    a = support(FX2,FX1,v)
    a,b,c,flag, _, _ = tf.while_loop(
        _cond_PickTriangle, _loop_PickTriangle, (a,b,c,flag, FX1, FX2), parallel_iterations=10, maximum_iterations=IterationAllowed
    )

    return a, b, c, flag

@tf.function
def _loop_pickTetrahedron(a, b, c, d, dist, flag, FX1, FX2):
    #Check the tetrahedron:
    ab = b-a
    ao = -a
    ac = c-a
    ad = d-a

    #We KNOW that the origin is not under the base of the tetrahedron based on
    #the way we picked a. So we need to check faces ABC, ABD, and ACD.

    #Normal to face of triangle
    abc = tf.linalg.cross(ab,ac)
    acd = tf.linalg.cross(ac,ad)
    adb = tf.linalg.cross(ad,ab)
    abco = K.sum(abc*ao)
    acdo = K.sum(acd*ao)
    adbo = K.sum(adb*ao)

    b,c,abc,flag = tf.cond(
        tf.greater(abco, 0), 
        lambda: (b,c,abc,flag), 
        lambda: tf.cond(
            tf.greater(acdo, 0),
            lambda: (c,d,acd,flag),
            lambda: tf.cond(
                tf.greater(adbo, 0),
                lambda: (d,b,adb,flag),
                lambda: (b,c,abc,True)
            )
        )
    )

    a,b,c,d = tf.cond(tf.greater(K.sum(abc*ao), 0), 
                        lambda: (support(FX2,FX1,abc),a,b,c), 
                        lambda: (support(FX2,FX1,-abc),a,c,b)
                       )
    return a, b, c, d, K.max([acdo, adbo]), flag, FX1, FX2

@tf.function
def _cond_pickTetrahedron(a, b, c, d, dist, flag, FX1, FX2):
    return tf.logical_not(flag)

@tf.function
def pickTetrahedronTF(a,b,c,FX1,FX2,IterationAllowed):
    flag = False
    ab = b-a
    ac = c-a

    # Normal to face of triangle
    abc = tf.linalg.cross(ab,ac)
    ao = -a

    a,b,c,d = tf.cond(tf.greater(K.sum(abc* ao), 0), 
                        lambda: (support(FX2,FX1,abc),a,b,c), 
                        lambda: (support(FX2,FX1,-abc),a,c,b)
                       )

    a, b, c, d, dist, flag, _, _ = tf.while_loop(
        _cond_pickTetrahedron, _loop_pickTetrahedron, (a,b,c,d,0.0,flag, FX1, FX2), 
        parallel_iterations=10, maximum_iterations=IterationAllowed
    )   
    return a,b,c,d,dist,flag

@tf.function
def test_collision(FX1, FX2, v, iterations):
    a, b = pickLineTF(v, FX2, FX1)
    a, b, c, flag = PickTriangleTF(a,b,FX2,FX1,iterations)
    a,b,c,d,dist,flag = tf.cond(flag, # Only bother if we could find a viable triangle.
                           lambda: pickTetrahedronTF(a,b,c,FX2,FX1,iterations),
                           lambda: (a,b,c,c,0.0,flag))
    return dist, flag

In [ ]:
gtimer.reset()
v = np.array([0.8000, 0.5000, 1.0000], dtype=np.float32)
t1 = time.time()
for _ in range(1000):
    dist, flag = test_collision(FX1, FX2, v, iterations)
    if flag:
        print("Collision")
#     else:
#         print("Distance: {}".fo)
t2 = time.time()
print("{} ms, while <1ms in matlab".format(t2-t1))
gtimer.print_time_log()

## batch version original

In [ ]:
BATCH_DIM_DEFAULT = 2

@tf.function
def getFarthestInDir_batch(FX_batch, v_batch, batch_dims=BATCH_DIM_DEFAULT):
    dotted = K.sum(FX_batch*v_batch,axis=-1)
    IdxSet = tf.expand_dims(tf.argmax(dotted, axis=-1),-1)
    point=tf.gather_nd(params = FX_batch, indices=IdxSet, batch_dims=batch_dims)
    return point

@tf.function
def support_batch(FX1_batch, FX2_batch, v_batch, batch_dims=BATCH_DIM_DEFAULT):
    point1 = getFarthestInDir_batch(FX1_batch, v_batch, batch_dims)
    point2 = getFarthestInDir_batch(FX2_batch, -v_batch, batch_dims)
    return point1 - point2

@tf.function
def pickLineTF_batch(v_batch, FX1_batch, FX2_batch, batch_dims=BATCH_DIM_DEFAULT):
    b= support_batch(FX2_batch, FX1_batch, v_batch, batch_dims)
    a= support_batch(FX2_batch, FX1_batch, -v_batch, batch_dims)
    return a, b

@tf.function
def _loop_PickTriangle_batch(a, b, c, flag, FX1_batch, FX2_batch):
    ab = b-a;
    ao = -a;
    ac = c-a;
    abc = tf.linalg.cross(ab,ac)
    abp = tf.linalg.cross(ab,abc)
    acp = tf.linalg.cross(abc,ac)
    abpo = tf.greater(K.sum(abp*ao,axis=-1, keepdims=True), 0)
    acpo = tf.greater(K.sum(acp*ao,axis=-1, keepdims=True), 0)
    abpo = tf.logical_and(tf.logical_not(flag),abpo)
    acpo = tf.logical_and(tf.logical_not(flag),tf.logical_and(tf.logical_not(abpo),acpo))
    def_case = tf.logical_not(tf.logical_or(abpo, acpo))
    flag = def_case
    abpo = tf.cast(abpo, tf.float32)
    acpo = tf.cast(acpo, tf.float32)
    def_case = tf.cast(def_case, tf.float32)
    abp = tf.expand_dims(abp, axis=-2)
    acp = tf.expand_dims(acp, axis=-2)
    c = abpo*b + acpo*c + def_case*c
    b = abpo*a + acpo*a + def_case*b
    a = abpo*support_batch(FX2_batch, FX1_batch, abp, BATCH_DIM_DEFAULT) + \
        acpo*support_batch(FX2_batch, FX1_batch, acp, BATCH_DIM_DEFAULT) + \
        def_case*a
    return a,b,c,flag, FX1_batch, FX2_batch

@tf.function
def _cond_PickTriangle_batch(a, b, c, flag, FX1_batch, FX2_batch):
    return tf.reduce_all(tf.logical_not(flag))
    

@tf.function
def PickTriangleTF_batch(a, b, FX1_batch, FX2_batch, IterationAllowed=6):
    flag = False

    # First try:
    ab = b-a
    ao = -a
    v_batch = tf.expand_dims(tf.linalg.cross(tf.linalg.cross(ab,ao),ab), axis=-2) # v is perpendicular to ab pointing in the general direction of the origin
    c = b
    b = a
    a = support_batch(FX2_batch,FX1_batch,v_batch,BATCH_DIM_DEFAULT)
    a,b,c,flag, _, _ = tf.while_loop(
        _cond_PickTriangle_batch, _loop_PickTriangle_batch, (a,b,c,FLAG_DEFAULT, FX1_batch, FX2_batch), 
        parallel_iterations=1, maximum_iterations=IterationAllowed
    )

    return a, b, c, flag

In [ ]:
@tf.function
def _loop_pickTetrahedron_batch(a, b, c, d, dist, flag, FX1_batch, FX2_batch):
    #Check the tetrahedron:
    ab = b-a
    ao = -a
    ac = c-a
    ad = d-a

    #We KNOW that the origin is not under the base of the tetrahedron based on
    #the way we picked a. So we need to check faces ABC, ABD, and ACD.

    #Normal to face of triangle
    abc = tf.linalg.cross(ab,ac)
    acd = tf.linalg.cross(ac,ad)
    adb = tf.linalg.cross(ad,ab)
    abco_val = K.sum(abc*ao, axis=-1, keepdims=True)
    acdo_val = K.sum(acd*ao, axis=-1, keepdims=True)
    adbo_val = K.sum(adb*ao, axis=-1, keepdims=True)
    dist = K.max([acdo_val, adbo_val],axis=0)
    abco = tf.greater(abco_val, 0)
    acdo = tf.greater(acdo_val, 0)
    adbo = tf.greater(adbo_val, 0)
    abco = tf.logical_and(tf.logical_not(flag),abco)
    flag_abco = tf.logical_or(flag,abco)
    acdo = tf.logical_and(tf.logical_not(flag_abco),acdo)
    flag_abco_acdo = tf.logical_or(flag_abco,acdo)
    adbo = tf.logical_and(tf.logical_not(flag_abco_acdo),adbo)
    def_case = tf.logical_not(tf.logical_or(tf.logical_or(abco, acdo), adbo))
    flag = def_case
    abco = tf.cast(abco, tf.float32)
    acdo = tf.cast(acdo, tf.float32)
    adbo = tf.cast(adbo, tf.float32)
    def_case = tf.cast(def_case, tf.float32)
    
    
    b,c = (abco*b+acdo*c+adbo*d+def_case*b, abco*c+acdo*d+adbo*b+def_case*c)
    abc = abco*abc+acdo*acd+adbo*adb+def_case*abc

    #abc = tf.expand_dims(abc, axis=-2)
    abco = tf.greater(K.sum(abc*ao), 0)
    abco = tf.cast(abco, tf.float32)
    abco_not = 1-abco
    d = abco*c + abco_not*b
    c = abco*b + abco_not*c
    b = a
    a = support_batch(FX2_batch,FX1_batch,tf.expand_dims(abco*abc+ abco_not*(-abc),axis=-2))
    return a, b, c, d, dist, flag, FX1_batch, FX2_batch

@tf.function
def _cond_pickTetrahedron_batch(a, b, c, d, dist, flag, FX1_batch, FX2_batch):
    return tf.reduce_all(tf.logical_not(flag))

@tf.function
def pickTetrahedronTF_batch(a,b,c,FX1_batch,FX2_batch,IterationAllowed):
    flag = False
    ab = b-a
    ac = c-a

    # Normal to face of triangle
    abc = tf.expand_dims(tf.linalg.cross(ab,ac),axis=-2)
    ao = -tf.expand_dims(a,axis=-2)

    a,b,c,d = tf.cond(tf.greater(K.sum(abc* ao), 0), 
                        lambda: (support_batch(FX2_batch,FX1_batch,abc),a,b,c), 
                        lambda: (support_batch(FX2_batch,FX1_batch,-abc),a,c,b)
                       )

    a, b, c, d, dist, flag, _, _ = tf.while_loop(
        _cond_pickTetrahedron_batch, _loop_pickTetrahedron_batch, 
        (a,b,c,d,DIST_DEFAULT,FLAG_DEFAULT, FX1_batch, FX2_batch), 
        parallel_iterations=10, maximum_iterations=IterationAllowed
    )   
    return a,b,c,d,dist,flag

In [ ]:
@tf.function
def test_collision_batch(FX1_batch, FX2_batch, v_batch, iterations):
    a, b = pickLineTF_batch(v_batch, FX2_batch, FX1_batch)
    a, b, c, flag = PickTriangleTF_batch(a,b,FX2_batch,FX1_batch,iterations)
    a,b,c,d,dist,flag = pickTetrahedronTF_batch(a,b,c,FX2_batch,FX1_batch,iterations)
    return dist, flag

## test batch

In [ ]:
N_sim = 50
N_col = 50
FLAG_DEFAULT = tf.constant([[[False]]*N_col]*N_sim)
DIST_DEFAULT = tf.constant([[[0.0]]*N_col]*N_sim)

In [ ]:
v_batch = np.array([[[[0.8000, 0.5000, 1.0000]]]*N_col]*N_sim,dtype=np.float32)
FX1_batch = tf.stack([tf.stack([FX1]*N_col, axis=0)]*N_sim,axis=0)
FX2_batch = tf.stack([tf.stack([FX2]*N_col, axis=0)]*N_sim,axis=0)
gtimer.reset()
for _ in range(100):
    gtimer.tic("TF_batch")
    dist, flag = test_collision_batch(FX1_batch, FX2_batch, v_batch, iterations)
    gtimer.toc("TF_batch")
gtimer.print_time_log()

In [ ]:
gtimer.reset()
for _ in range(1000):
    gtimer.tic("FD")
    dist, flag = test_collision(FX1, FX2, v, iterations)
    gtimer.toc("FD")
gtimer.print_time_log()

In [ ]:
class CollisionLayer(layers.Layer):
    def __init__(self,*args, **kwargs):
        super(CollisionLayer, self).__init__(*args, **kwargs)
        
#     # 변수를 만듭니다.
#     def build(self, input_shape):
#         pass

    # call 메서드가 그래프 모드에서 사용되면
    # training 변수는 텐서가 됩니다.
    @tf.function
    def call(self, inputs=None):
        v = tf.constant([0.8000, 0.5000, 1.0000])
        FX1 = inputs[0]
        FX2 = inputs[1]
        return test_collision(FX1, FX2, v, iterations)

In [ ]:

input = np.stack((FX1, FX2),axis=0)
print(input.shape)
cl = CollisionLayer()
cl(input)

In [ ]:
inputs = tf.keras.Input(shape=(10, 128, 128, 3))
conv_2d_layer = tf.keras.layers.Conv2D(64, (3, 3))
outputs = tf.keras.layers.TimeDistributed(conv_2d_layer)(inputs)
outputs.shape

In [ ]:
input_val = np.random.rand(1,10,128,128,3).astype('float32')
conv_2d_layer = tf.keras.layers.Conv2D(64, (3, 3))
td_conv = tf.keras.layers.TimeDistributed(conv_2d_layer)
gtimer = GlobalTimer()
for _ in range(10):
    gtimer.tic("td")
    outputs = td_conv(input_val)
    gtimer.toc("td")
gtimer.print_time_log()

input_val = np.random.rand(100,10,128,128,3).astype('float32')
conv_2d_layer = tf.keras.layers.Conv2D(64, (3, 3))
td_conv = tf.keras.layers.TimeDistributed(conv_2d_layer)
gtimer = GlobalTimer()
for _ in range(10):
    gtimer.tic("td")
    outputs = td_conv(input_val)
    gtimer.toc("td")
gtimer.print_time_log()

# Matthew Sheen version

In [ ]:
import time
import collections
class GlobalTimer:
    def __init__(self, scale=1000):
        self.name_list = []
        self.ts_dict = {}
        self.time_dict = collections.defaultdict(lambda: 0)
        self.count_dict = collections.defaultdict(lambda: 0)
        self.scale = scale
        self.switch(True)
        
    def reset(self):
        self.name_list = []
        self.ts_dict = {}
        self.time_dict = collections.defaultdict(lambda: 0)
        self.count_dict = collections.defaultdict(lambda: 0)
        self.switch(True)
        
    def switch(self, onoff):
        self.__on = onoff
    
    def tic(self, name):
        if self.__on:
            if name not in self.name_list:
                self.name_list += [name]
            self.ts_dict[name] = time.time()
        
    def toc(self, name):
        if self.__on:
            self.time_dict[name] = self.time_dict[name]+(time.time() - self.ts_dict[name]) * self.scale
            self.count_dict[name] = self.count_dict[name] + 1
            
    def toctic(self, name_toc, name_tic):
        self.toc(name_toc)
        self.tic(name_tic)
        
    def print_time_log(self, names=None, timeunit="ms"):
        if names is None:
            names = self.name_list
        for name in names:
            print("{name}: \t{tot_T} {timeunit}/{tot_C} = {per_T} {timeunit}".format(
                name=name, tot_T=np.round(np.sum(self.time_dict[name])), tot_C=self.count_dict[name], 
                per_T= np.round(np.sum(self.time_dict[name])/self.count_dict[name], 3),
                timeunit=timeunit
            ))
        

In [ ]:
gtimer = GlobalTimer()

In [ ]:
global_save = {}
def getFarthestInDir(FX, v):
    global_save['FX'] = FX
    global_save['v'] = v
    gtimer.tic("getFarthestInDir")
    gtimer.tic("getFarthestInDir0")
    dotted = K.sum(tf.multiply(FX,v),axis=-1)
    gtimer.toctic("getFarthestInDir0", "getFarthestInDir1")
    rowIdxSet = tf.argmax(dotted, axis=-1)
    gtimer.toctic("getFarthestInDir1","getFarthestInDir2")
    maxInCol = K.max(dotted, axis=-1)
    gtimer.toctic("getFarthestInDir2", "getFarthestInDir3")
    colIdx = tf.expand_dims(tf.argmax(maxInCol, axis=-1),axis=-1)
    gtimer.toctic("getFarthestInDir3", "getFarthestInDir4")
    rowIdx=tf.gather(params = rowIdxSet, indices=colIdx)
    gtimer.toctic("getFarthestInDir4", "getFarthestInDir5")
    point=tf.gather_nd(params = FX, indices=tf.concat([colIdx,rowIdx],axis=-1))
    gtimer.toc("getFarthestInDir5")
    gtimer.toc("getFarthestInDir")
    return point

def get_facepoints(verts, faces): # (fc, vtx, ax)
    return np.array([verts[s1f-1] for s1f in faces])

def support(FX1, FX2, v):
    gtimer.tic("support")
    point1 = getFarthestInDir(FX1, v)
    point2 = getFarthestInDir(FX2, -v)
    gtimer.toc("support")
    return point1 - point2

def pickLine(v, FX1, FX2):
    gtimer.tic("pickLine")
    b= support(FX2, FX1, v)
    a= support(FX2, FX1, -v)
    gtimer.toc("pickLine")
    return a, b

def PickTriangle(a, b, FX1, FX2, IterationAllowed=6):
    flag = 0

    gtimer.tic("PickTriangle_init")
    # First try:
    ab = b-a
    ao = -a
    v = tf.linalg.cross(tf.linalg.cross(ab,ao),ab) # v is perpendicular to ab pointing in the general direction of the origin
    c = b
    b = a
    a = support(FX2,FX1,v)
    gtimer.toc("PickTriangle_init")
    for i in range(IterationAllowed):
        gtimer.tic("PickTriangle_loop")
        gtimer.tic("PickTriangle_loop0")
        ab = b-a;
        ao = -a;
        ac = c-a;
        abc = tf.linalg.cross(ab,ac)
        abp = tf.linalg.cross(ab,abc)
        acp = tf.linalg.cross(abc,ac)
        gtimer.toc("PickTriangle_loop0")
        if K.sum(abp*ao) > 0:
            c = b # Throw away the furthest point and grab a new one in the right direction
            b = a
            v = abp # cross(cross(ab,ao),ab);
        elif K.sum(acp*ao) > 0:
            b = a
            v = acp; # cross(cross(ac,ao),ac);

        else:
            flag = 1;
            break # We got a good one.
        a = support(FX2, FX1,v)
        gtimer.toc("PickTriangle_loop")

    return a, b, c, flag


def pickTetrahedron(a,b,c,FX1,FX2,IterationAllowed):
    gtimer.tic("pickTetrahedron_init")
    flag = 0
    ab = b-a
    ac = c-a

    # Normal to face of triangle
    gtimer.tic("pickTetrahedron_init0")
    abc = tf.linalg.cross(ab,ac)
    gtimer.toc("pickTetrahedron_init0")
    ao = -a

    gtimer.tic("pickTetrahedron_init1")
    if K.sum(abc* ao) > 0: # Above
        d = c
        c = b
        b = a

        v = abc
        a = support(FX2,FX1,v) # Tetrahedron new point
    else: # below
        d = b
        b = a
        v = -abc
        a = support(FX2,FX1,v) # Tetrahedron new point
    gtimer.toc("pickTetrahedron_init1")
    gtimer.toc("pickTetrahedron_init")

    for i in range(IterationAllowed): #Allowing 10 tries to make a good tetrahedron.
        gtimer.tic("pickTetrahedron_loop")
        gtimer.tic("pickTetrahedron_loop0")
        #Check the tetrahedron:
        ab = b-a
        ao = -a
        ac = c-a
        ad = d-a

        #We KNOW that the origin is not under the base of the tetrahedron based on
        #the way we picked a. So we need to check faces ABC, ABD, and ACD.

        #Normal to face of triangle
        abc = tf.linalg.cross(ab,ac)

        if K.sum(abc*ao) > 0: #Above triangle ABC
            pass
            # No need to change anything, we'll just iterate again with this face as
            # default.
        else:
            acd = tf.linalg.cross(ac,ad) # Normal to face of triangle

            if K.sum(acd*ao) > 0 : # Above triangle ACD
                # Make this the new base triangle.
                b = c
                c = d
                ab = ac
                ac = ad            
                abc = acd
            elif K.sum(acd*ao) < 0:
                adb = tf.linalg.cross(ad,ab) #Normal to face of triangle

                if K.sum(adb*ao) > 0: #Above triangle ADB
                    # Make this the new base triangle.
                    c = b
                    b = d
                    ac = ab
                    ab = ad
                    abc = adb
                else:
                    flag = 1
                    break # It's inside the tetrahedron.
        gtimer.toc("pickTetrahedron_loop0")

        #try again:
        if K.sum(abc*ao) > 0: #Above
            d = c
            c = b
            b = a    
            v = abc
            a = support(FX2,FX1,v) #Tetrahedron new point
        else: #below
            d = b;
            b = a;
            v = -abc;
            a = support(FX2,FX1,v) #Tetrahedron new point
        gtimer.toc("pickTetrahedron_loop")
    return a,b,c,d,flag

# Test

In [ ]:
gtimer = GlobalTimer()

In [ ]:
s1coords = np.loadtxt("s1.csv", delimiter=",")
s2coords = np.loadtxt("s2.csv", delimiter=",")

In [ ]:
s1faces = np.loadtxt("s1face.csv", delimiter=",",dtype='int')
s2faces = np.loadtxt("s2face.csv", delimiter=",",dtype='int')

In [ ]:
iterations = 6

In [ ]:
s1verts = get_facepoints(s1coords, s1faces)
s2verts = get_facepoints(s2coords, s2faces)

In [ ]:
S1Rot = [
        [0.99847,0.016541,-0.052829,],
        [-0.014912,0.99941,0.031073,],
        [0.053311,-0.030238,0.99812,],
        ]
S2Rot = [
        [0.99752,0.065339,-0.026294,],
        [-0.063664,0.99616,0.060141,],
        [0.030123,-0.058318,0.99784,],
        ]

In [ ]:
s1points = tf.matmul(s1verts[:,:,:], np.transpose(S1Rot))+3
s2points = tf.matmul(s2verts[:,:,:], np.transpose(S2Rot))-3

In [ ]:
FX1 = s1points
FX2 = s2points

In [ ]:
gtimer.reset()
v = np.array([0.8000, 0.5000, 1.0000])
t1 = time.time()
for _ in range(1000):
    a, b = pickLine(v, FX2, FX1)
    a, b, c, flag = PickTriangle(a,b,FX2,FX1,iterations)
    if flag == 1: # Only bother if we could find a viable triangle.
        a,b,c,d,flag = pickTetrahedron(a,b,c,FX2,FX1,iterations)
t2 = time.time()
print("{} ms, while <1ms in matlab".format(t2-t1))

In [ ]:
v = np.array([0.8000, 0.5000, 1.0000])
a,b = pickLine(v, FX2, FX1)
a,b

In [ ]:
a, b, c, flag = PickTriangle(a,b,FX2,FX1,iterations)
a, b, c

In [ ]:
pickTetrahedron(a,b,c,FX2,FX1,iterations)

# Matthew Sheen numpy version

In [ ]:
def getFarthestInDir(FX, v):
    dotted = np.sum(FX*v,axis=-1)
    rowIdxSet = np.argmax(dotted, axis=-1)
    maxInCol = np.max(dotted, axis=-1)
    colIdx = np.argmax(maxInCol, axis=-1)
    rowIdx = rowIdxSet[colIdx]
    point = FX[colIdx,rowIdx]
    return point


def get_facepoints(verts, faces): # (fc, vtx, ax)
    return np.array([verts[s1f-1] for s1f in faces])

def support(FX1, FX2, v):
    point1 = getFarthestInDir(FX1, v)
    point2 = getFarthestInDir(FX2, -v)
    return point1 - point2

def pickLine(v, FX1, FX2):
    b= support(FX2, FX1, v)
    a= support(FX2, FX1, -v)
    return a, b

def PickTriangle(a, b, FX1, FX2, IterationAllowed=6):
    flag = 0

    # First try:
    ab = b-a
    ao = -a
    v = np.cross(np.cross(ab,ao),ab) # v is perpendicular to ab pointing in the general direction of the origin
    c = b
    b = a
    a = support(FX2,FX1,v)
    for i in range(IterationAllowed):
        ab = b-a;
        ao = -a;
        ac = c-a;
        abc = np.cross(ab,ac)
        abp = np.cross(ab,abc)
        acp = np.cross(abc,ac)
        if np.sum(abp*ao) > 0:
            c = b # Throw away the furthest point and grab a new one in the right direction
            b = a
            v = abp # cross(cross(ab,ao),ab);
        elif np.sum(acp*ao) > 0:
            b = a
            v = acp; # cross(cross(ac,ao),ac);

        else:
            flag = 1;
            break # We got a good one.
        a = support(FX2, FX1,v)

    return a, b, c, flag


def pickTetrahedron(a,b,c,FX1,FX2,IterationAllowed):
    flag = 0
    ab = b-a
    ac = c-a

    # Normal to face of triangle
    abc = np.cross(ab,ac)
    ao = -a

    if np.sum(abc* ao) > 0: # Above
        d = c
        c = b
        b = a

        v = abc
        a = support(FX2,FX1,v) # Tetrahedron new point
    else: # below
        d = b
        b = a
        v = -abc
        a = support(FX2,FX1,v) # Tetrahedron new point

    for i in range(IterationAllowed): #Allowing 10 tries to make a good tetrahedron.
        #Check the tetrahedron:
        ab = b-a
        ao = -a
        ac = c-a
        ad = d-a

        #We KNOW that the origin is not under the base of the tetrahedron based on
        #the way we picked a. So we need to check faces ABC, ABD, and ACD.

        #Normal to face of triangle
        abc = np.cross(ab,ac)

        if np.sum(abc*ao) > 0: #Above triangle ABC
            pass
            # No need to change anything, we'll just iterate again with this face as
            # default.
        else:
            acd = np.cross(ac,ad) # Normal to face of triangle

            if np.sum(acd*ao) > 0 : # Above triangle ACD
                # Make this the new base triangle.
                b = c
                c = d
                ab = ac
                ac = ad            
                abc = acd
            elif np.sum(acd*ao) < 0:
                adb = np.cross(ad,ab) #Normal to face of triangle

                if np.sum(adb*ao) > 0: #Above triangle ADB
                    # Make this the new base triangle.
                    c = b
                    b = d
                    ac = ab
                    ab = ad
                    abc = adb
                else:
                    flag = 1
                    break # It's inside the tetrahedron.

        #try again:
        if np.sum(abc*ao) > 0: #Above
            d = c
            c = b
            b = a    
            v = abc
            a = support(FX2,FX1,v) #Tetrahedron new point
        else: #below
            d = b;
            b = a;
            v = -abc;
            a = support(FX2,FX1,v) #Tetrahedron new point
    return a,b,c,d,flag

# Test

In [ ]:
s1coords = np.loadtxt("s1.csv", delimiter=",")
s2coords = np.loadtxt("s2.csv", delimiter=",")

In [ ]:
s1faces = np.loadtxt("s1face.csv", delimiter=",",dtype='int')
s2faces = np.loadtxt("s2face.csv", delimiter=",",dtype='int')

In [ ]:
iterations = 6

In [ ]:
s1verts = get_facepoints(s1coords, s1faces)
s2verts = get_facepoints(s2coords, s2faces)

In [ ]:
S1Rot = [
        [0.99847,0.016541,-0.052829,],
        [-0.014912,0.99941,0.031073,],
        [0.053311,-0.030238,0.99812,],
        ]
S2Rot = [
        [0.99752,0.065339,-0.026294,],
        [-0.063664,0.99616,0.060141,],
        [0.030123,-0.058318,0.99784,],
        ]

In [ ]:
s1points = tf.matmul(s1verts[:,:,:], np.transpose(S1Rot))+3
s2points = tf.matmul(s2verts[:,:,:], np.transpose(S2Rot))-3

In [ ]:
FX1 = s1points
FX2 = s2points

In [ ]:
t1 = time.time()
for _ in range(1000):
    v = np.array([0.8000, 0.5000, 1.0000])
    a, b = pickLine(v, FX2, FX1)
    a, b, c, flag = PickTriangle(a,b,FX2,FX1,iterations)
    if flag == 1: # Only bother if we could find a viable triangle.
        a,b,c,d,flag = pickTetrahedron(a,b,c,FX2,FX1,iterations)
t2 = time.time()
print("{} ms, while <1ms in matlab".format(t2-t1))